# Word2Vec（単語のベクトル化）

## 学習用のデータを準備する（コーパス）

In [ ]:
# # プロジェクトカタログを読み込む
# project_df = pd.read_csv('../data/raw/project.csv')
# project_df.head()

,id,name,description,desired_role,start_date,end_date,HTML,CSS,JavaScript,TypeScript,...,GCP,Docker,Kubernetes,MySQL,PostgreSQL,MongoDB,Git,Linux,Agile,Scrum
0,1,旅行予約システム設計,当プロジェクトでは運用保守、テスト、デプロイ、ドキュメント作成、自動テスト導入やインフラ構築...,QAエンジニア,2024-04-09,NaN,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0
1,2,予約管理システム設計,現プロジェクトにおいてデータベース設計やデプロイなどに取り組んでいる。\n[実施内容]\nデ...,フルスタックエンジニア,2024-12-22,NaN,1,1,1,1,...,0,1,1,0,0,0,1,1,0,0
2,3,ブロックチェーン開発実装,要件定義、要件分析、パフォーマンス改善、自動テスト導入、クラウド移行やセキュリティ強化などの...,フルスタックエンジニア,2025-05-20,NaN,1,1,1,0,...,0,0,0,1,1,1,1,1,0,0
3,4,動画配信サービス改善,技術選定、UI/UX設計、セキュリティ強化、実装、デプロイやデータベース設計などの開発に取り...,インフラエンジニア,2024-11-24,NaN,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
4,5,物流管理システム改善,データベース設計、運用保守やチームマネジメントなどの開発に取り組んでいます。,SE,2024-02-22,NaN,0,0,0,0,...,0,0,0,1,1,0,1,1,1,0


In [ ]:
# # 全テキストから全ての単語を抽出してWord2Vec用のコーパスを作成
# def create_corpus(df, columns):
#     """
#     DataFrameから指定列のテキストを抽出し、全単語のリストのリストを作成
#     """
#     corpus = []
    
#     for _, row in df.iterrows():
#         # 各行の指定列を結合
#         combined_text = ' '.join([str(row[col]) for col in columns if pd.notna(row[col])])
        
#         # 形態素解析を実行
#         tokens = tokenizer_obj.tokenize(combined_text)
        
#         # 全ての表層形を抽出
#         words = [token.surface() for token in tokens]
        
#         if words:  # 空でない場合のみ追加
#             corpus.append(words)
    
#     return corpus

# # ユーザー履歴のコーパスを作成
# user_columns = ['project_name', 'description', 'role']
# user_corpus = create_corpus(df, user_columns)

# # プロジェクトカタログのコーパスを作成
# project_columns = ['name', 'description', 'desired_role']
# project_corpus = create_corpus(project_df, project_columns)

# # 両方を結合
# corpus = user_corpus + project_corpus

# print(f"ユーザー職歴コーパス: {len(user_corpus)}件")
# print(f"プロジェクトカタログコーパス: {len(project_corpus)}件")
# print(f"結合コーパス: {len(corpus)}件")

# print("\n【ユーザー職歴】最初の2件のサンプル:")
# for i, doc in enumerate(user_corpus[:2]):
#     print(f"{i+1}: {doc[:10]}...")

# print("\n【プロジェクトカタログ】最初の2件のサンプル:")
# for i, doc in enumerate(project_corpus[:2]):
#     print(f"{i+1}: {doc[:10]}...")

## 全テキストから名詞のみを抽出してWord2Vec用のコーパスを作成

In [ ]:
# 全テキストから名詞のみを抽出してWord2Vec用のコーパスを作成
def create_corpus(df, columns):
    """
    DataFrameから指定列のテキストを抽出し、名詞のみのリストのリストを作成
    """
    corpus = []
    
    for _, row in df.iterrows():
        # 各行の指定列を結合
        combined_text = ' '.join([str(row[col]) for col in columns if pd.notna(row[col])])
        
        # 形態素解析を実行
        tokens = tokenizer_obj.tokenize(combined_text)
        
        # 名詞のみを抽出
        words = [token.surface() for token in tokens if token.part_of_speech()[0] == '名詞']
        
        if words:  # 空でない場合のみ追加
            corpus.append(words)
    
    return corpus

# ユーザー履歴のコーパスを作成
user_columns = ['project_name', 'description', 'role']
user_corpus = create_corpus(df, user_columns)

# プロジェクトカタログのコーパスを作成
# project_columns = ['name', 'description', 'desired_role']
# project_corpus = create_corpus(project_df, project_columns)

# 両方を結合
# corpus = user_corpus + project_corpus
corpus = user_corpus

print(f"ユーザー職歴コーパス: {len(user_corpus)}件")
print(f"プロジェクトカタログコーパス: {len(project_corpus)}件")
print(f"結合コーパス: {len(corpus)}件")

print("\n【ユーザー職歴】最初の2件のサンプル:")
for i, doc in enumerate(user_corpus[:2]):
    print(f"{i+1}: {doc[:10]}...")

# print("\n【プロジェクトカタログ】最初の2件のサンプル:")
# for i, doc in enumerate(project_corpus[:2]):
#     print(f"{i+1}: {doc[:10]}...")

ユーザー職歴コーパス: 911件
プロジェクトカタログコーパス: 1000件
結合コーパス: 911件

【ユーザー職歴】最初の2件のサンプル:
1: ['スマホアプリ', '開発', 'SNS', 'アプリ', '開発', '担当', 'ドキュメント', '作成', 'テスト', '実施']...
2: ['医療情報システム', '開発', 'オンプレミス', '環境', 'AWS', 'クラウド', '移行', '主導', '新技術', '調査']...

【プロジェクトカタログ】最初の2件のサンプル:
1: ['旅行', '予約', 'システム設計', 'プロジェクト', '運用', '保守', 'テスト', 'デプロイ', 'ドキュメント', '作成']...
2: ['予約', '管理', 'システム設計', 'プロジェクト', 'データベース設計', 'デプロイ', '実施', '内容', 'データベース設計', 'デプロイ']...


## モデルの学習

In [ ]:
# Word2Vecモデルの学習
model = Word2Vec(
    sentences=corpus,      # 学習データ
    vector_size=100,       # ベクトルの次元数
    window=5,              # コンテキストウィンドウのサイズ
    min_count=1,           # 最小出現回数（これ以下の単語は無視）
    # workers=4,             # 並列処理のワーカー数
    sg=1,                  # 0: CBOW, 1: Skip-gram
    epochs=100,             # エポック数
    negative=5,            # ネガティブサンプリング数（Skip-gram使用時）
    alpha=0.025,           # 初期学習率（デフォルト値）
    min_alpha=0.0001,      # 最小学習率
    seed=42                # 再現性のため
)

print(f"語彙数: {len(model.wv)}語")
print(f"ベクトルの次元数: {model.wv.vector_size}")

語彙数: 140語
ベクトルの次元数: 100


In [ ]:
# 語彙数の確認
print(f"語彙数: {len(model.wv)}語")

# 頻出単語の確認
all_words = [word for doc in corpus for word in doc]
word_freq = Counter(all_words)
print(f"\n総単語数: {len(all_words)}")
print(f"ユニーク単語数: {len(word_freq)}")
print("\n頻出単語 Top 20:")
for word, count in word_freq.most_common(20):
    print(f"  {word}: {count}回")

語彙数: 140語

総単語数: 18275
ユニーク単語数: 140

頻出単語 Top 20:
  担当: 805回
  開発: 799回
  テスト: 470回
  構築: 444回
  設計: 412回
  改善: 332回
  クラウド: 282回
  チーム: 282回
  プロジェクト: 254回
  UI: 222回
  UX: 222回
  エンジニア: 222回
  レビュー: 211回
  顧客: 211回
  調整: 211回
  実施: 208回
  リファクタリング: 207回
  既存: 204回
  システム: 204回
  要件: 202回


In [ ]:
# モデルの保存
model.save("../results/models/word2vec_model.bin")
print("モデルを保存しました: word2vec_model.bin")

# 単語ベクトルのみを保存（より軽量）
model.wv.save("../results/models/word2vec_vectors.kv")
print("単語ベクトルを保存しました: word2vec_vectors.kv")

モデルを保存しました: word2vec_model.bin
単語ベクトルを保存しました: word2vec_vectors.kv


In [ ]:
# 単語ベクトルの確認
test_words = ['開発', 'システム', 'AWS', 'React', 'Python']

for word in test_words:
    if word in model.wv:
        vector = model.wv[word]
        print(f"\n単語: {word}")
        print(f"ベクトル (最初の10次元): {vector[:10]}")
        print(f"ベクトルの形状: {vector.shape}")
    else:
        print(f"\n単語 '{word}' は語彙に含まれていません")


単語: 開発
ベクトル (最初の10次元): [-0.11796082 -0.16586249  0.15224838 -0.28193828 -0.20976922  0.284552
  0.00346189  0.06835559  0.18295953 -0.16947858]
ベクトルの形状: (100,)

単語: システム
ベクトル (最初の10次元): [-0.13215175 -0.12867358  0.18591525 -0.40815824  0.20622589  0.22197193
  0.2578488   0.32634795 -0.13182755 -0.06729237]
ベクトルの形状: (100,)

単語: AWS
ベクトル (最初の10次元): [-0.07182981 -0.24260373 -0.5322119   0.3196711   0.0406214  -0.30522817
 -0.19449857  0.36219177 -0.5959646   0.11270412]
ベクトルの形状: (100,)

単語: React
ベクトル (最初の10次元): [-0.10373149  0.23902212 -0.3685489   0.33588377 -0.22443077  0.15456818
 -0.06230635  0.22421545 -0.06158894  0.61248684]
ベクトルの形状: (100,)

単語 'Python' は語彙に含まれていません


In [ ]:
# 類似単語の検索
test_word = 'AWS'

if test_word in model.wv:
    similar_words = model.wv.most_similar(test_word, topn=10)
    print(f"\n'{test_word}' に類似する単語:")
    for word, similarity in similar_words:
        print(f"  {word}: {similarity:.4f}")
else:
    print(f"単語 '{test_word}' は語彙に含まれていません")


'AWS' に類似する単語:
  主導: 0.5236
  オンプレミス: 0.4971
  環境: 0.4639
  デプロイ: 0.4100
  CD: 0.3938
  CI: 0.3930
  パイプライン: 0.3929
  調査: 0.3826
  課題: 0.3818
  管理: 0.3804


In [ ]:
# 類似単語の検索
test_word = 'リーダー'

if test_word in model.wv:
    similar_words = model.wv.most_similar(test_word, topn=10)
    print(f"\n'{test_word}' に類似する単語:")
    for word, similarity in similar_words:
        print(f"  {word}: {similarity:.4f}")
else:
    print(f"単語 '{test_word}' は語彙に含まれていません")


'リーダー' に類似する単語:
  テスター: 0.6185
  プログラマー: 0.5881
  SE: 0.5686
  SWE: 0.5564
  PM: 0.5530
  PG: 0.5228
  インフラエンジニア: 0.4912
  システムエンジニア: 0.4896
  デザイナー: 0.4676
  QA: 0.4140


In [ ]:
# 単語間の類似度を計算
word_pairs = [
    ('AWS', 'Azure'),
    ('React', 'Vue'),
    ('Python', 'Java'),
    ('開発', '構築'),
    ('プログラミング', '開発'),
    ('プログラミング', '構築'),
]

print("\n単語間の類似度:")
for word1, word2 in word_pairs:
    if word1 in model.wv and word2 in model.wv:
        similarity = model.wv.similarity(word1, word2)
        print(f"  {word1} - {word2}: {similarity:.4f}")
    else:
        print(f"  {word1} - {word2}: どちらかの単語が語彙に含まれていません")


単語間の類似度:
  AWS - Azure: どちらかの単語が語彙に含まれていません
  React - Vue: 0.5326
  Python - Java: どちらかの単語が語彙に含まれていません
  開発 - 構築: 0.1938
  プログラミング - 開発: 0.2018
  プログラミング - 構築: 0.2672


In [ ]:
# 全単語のベクトルをDataFrameに変換して保存
word_vectors_df = pd.DataFrame(
    [model.wv[word] for word in model.wv.index_to_key],
    index=model.wv.index_to_key
)

# CSVに保存
word_vectors_df.to_csv('../data/external/word_vectors.csv')
print("\n単語ベクトルをCSVに保存しました: data/external/word_vectors.csv")
print(f"形状: {word_vectors_df.shape}")
print("\n最初の5単語:")
print(word_vectors_df.head())


単語ベクトルをCSVに保存しました: data/external/word_vectors.csv
形状: (140, 100)

最初の5単語:
           0         1         2         3         4         5         6   \
担当   0.253113  0.142081  0.107371 -0.036470  0.329581  0.090653 -0.131531   
開発  -0.117961 -0.165862  0.152248 -0.281938 -0.209769  0.284552  0.003462   
テスト  0.081841  0.038523  0.276648  0.032594 -0.001037  0.324496  0.165522   
構築  -0.212445  0.108158 -0.231526 -0.123558  0.415946  0.014031 -0.121279   
設計   0.455223  0.096547  0.037000  0.293948  0.272290  0.063192 -0.009637   

           7         8         9   ...        90        91        92  \
担当  -0.060467 -0.106479 -0.214658  ...  0.016891  0.026645  0.222169   
開発   0.068356  0.182960 -0.169479  ...  0.077064 -0.133742  0.037159   
テスト  0.370206 -0.188463  0.180798  ...  0.062310 -0.216112 -0.085923   
構築  -0.123835 -0.401719 -0.045302  ... -0.204370  0.166554 -0.017961   
設計  -0.227307  0.148356  0.002225  ... -0.173117  0.283950  0.178416   

           93        94      

## ベクトルの図示